# Numerical Method HW 5
# Hannah Wang

## Question 1 

### 1(a) 

As T2-delivery forward price Ft: a forward contract paying ST2 − Ft at time T2 has time-t value 0, then

$$S_{T_2}-F_t = S_0 - e^{-r (T_2-t)} F_t$$
$$S_{T_2}-K = S_0 - e^{-r (T_2-t)}K $$

Thus,

$$ f_t = (S_{T_2}-K) - (S_{T_2}-F_t) = e^{-r (T_2-t)} (F_t-K) $$

### 1(b)

For crude oil, it will require the warehousing cost and the delivery cost. It is not frictionless.

### 1(c) 1(d)

In [82]:
import numpy as np

In [83]:
# Exponential Ornstein-Uhlenbeck process

class XOU:
    
    def __init__(self, kappa, alpha, sigma, S0, r):
        
        self.kappa = kappa
        self.alpha = alpha
        self.sigma = sigma
        self.S0 = S0
        self.r = r

In [84]:
hw5dynamics=XOU(kappa = 0.472, alpha = 4.4, sigma = 0.368, S0 = 106.9, r = 0.05)

In [85]:
class CallOnForwardPrice:
    
    def __init__(self, K1, T1, T2):
        
        self.K1 = K1
        self.T1 = T1
        self.T2 = T2
        

In [86]:
hw5contract=CallOnForwardPrice(K1 = 103.2, T1 = 0.5, T2 = 0.75)

In [1]:
class MC:
    
    def __init__(self, N, M, epsilon, seed):
        
        self.N = N   # Number of timesteps on each path
        self.M = M   # Number of paths
        self.epsilon = epsilon  # For the dC/dS calculation
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
    
    def price_call_XOU(self, contract, dynamics):
            
        # You complete the coding of this function
        # self.rng.normal() generates pseudo-random normals
        S0, kappa, alpha, r, sigma = dynamics.S0, dynamics.kappa, dynamics.alpha, dynamics.r, dynamics.sigma
        T1, T2, K1 = contract.T1, contract.T2, contract.K1
        
        deltat = T1 / self.N
        sigma2 = sigma*np.sqrt(deltat)
        
        X = np.zeros((self.M, self.N+1))
        X[:,0] = np.ones(self.M)*np.log(S0)
        X2 = np.zeros((self.M, self.N+1))
        X2[:,0] = np.ones(self.M)*np.log(S0+self.epsilon)
        
        for i in range(self.N):
            dW = np.random.randn(self.M)
            X_t = X[:,i]
            X[:,i+1] = X_t + kappa*(alpha-X_t)*deltat + sigma2*dW
            X_t2 = X2[:,i]
            X2[:,i+1] = X_t2 + kappa*(alpha-X_t2)*deltat + sigma2*dW
    
        S = np.exp(X)
        t = np.linspace(0, T1, self.N+1)
        F = np.exp(np.exp(-kappa * (T2-t))*X + (1-np.exp(-kappa * (T2-t)))*alpha + sigma**2/(4*kappa)*(1-np.exp(-kappa * (T2-t))**2))
        C = np.exp(-r*T1) * np.maximum(F[:,-1] - K1, 0)
        call_price = np.mean(C)
        standard_error = np.std(C, ddof=1) / np.sqrt(self.M)
    
        F2 = np.exp(np.exp(-kappa * (T2-t))*X2 + (1-np.exp(-kappa * (T2-t)))*alpha + sigma**2/(4*kappa)*(1-np.exp(-kappa * (T2-t))**2))
        C2 = np.exp(-r*T1) * np.maximum(F2[:,-1] - K1, 0)
        call_delta = (np.mean(C2) - call_price) / self.epsilon
        
        return(call_price, standard_error, call_delta)
        

In [88]:
hw5MC = MC(N=100, M=100000, epsilon=0.01, seed=0)
# Change M if necessary

In [89]:
(call_price, standard_error, call_delta) = hw5MC.price_call_XOU(hw5contract,hw5dynamics)

In [90]:
print(call_price, standard_error, call_delta)

7.794311442516019 0.042282442513031954 0.34128845280054776


### 1(e) 

We know that

$ f_t = e^{-r (T_2-t)} (F_t-K) $

$ F_t = exp \bigg[ e^{-\kappa (T_2-t)} \log S_t + (1-e^{-\kappa (T_2-t)})\alpha + \frac{\sigma^2}{4\kappa} (1-e^{-2\kappa (T_2-t)}) \bigg]$

Then,

$ \frac{\partial F_0}{\partial S} = e^{-\kappa T_2} S_t^{e^{-\kappa T_2} -1} exp \bigg[ (1-e^{-\kappa T_2})\alpha + \frac{\sigma^2}{4\kappa} (1-e^{-2\kappa T_2}) \bigg] $

$ \frac{\partial f_0}{\partial S}$
$= e^{-r T_2} \frac{\partial F_0}{\partial S}$
$= S_0^{e^{-\kappa T_2} -1} exp \bigg[ T_2 (-r-\kappa) + (1-e^{-\kappa (T_2)})\alpha + \frac{\sigma^2}{4\kappa} (1-e^{-2\kappa (T_2)}) \bigg]$

In [91]:
T2 = hw5contract.T2
kappa, alpha, sigma, S0, r = hw5dynamics.kappa, hw5dynamics.alpha, hw5dynamics.sigma, hw5dynamics.S0, hw5dynamics.r

forward_delta = S0**(np.exp(-kappa*T2)-1) * np.exp(T2 * (-r-kappa) + (1-np.exp(-kappa*T2))*alpha + sigma**2/(4*kappa)*(1-np.exp(-2*kappa*T2)))
print(forward_delta)

0.6465109164314246


### 1(f) 

In [92]:
print(call_delta/forward_delta)

0.5278927921037637


### 1(g)

In [93]:
T1, K1 = hw5contract.T1, hw5contract.K1

F_0 = np.exp(np.exp(-kappa*(T2))*np.log(S0) + (1-np.exp(-kappa*(T2)))*alpha + sigma**2/(4*kappa)*(1-np.exp(-2*kappa*(T2))))
f_0 = np.exp(-r*(T2)) * (F_0-K1)

contract_price = 4000*f_0 + 1000*call_price
print(contract_price)

4058.4742918982947


## Question 2 

### 2(a) 

Find the expectation and standard deviation of the fraction of the pot that Patrik will collect, when the last card (the “river”) is dealt in the usual way.

$$ E[P] = \frac{34}{44} = 0.7727$$

$$ Std[P] = \sqrt {Var[P]} = \sqrt {E[P^2] - E[P]^2} = \sqrt {0.1756} = 0.4191$$

### 2(b) 

With replacement,

$$ E[3P_i] = \frac{1}{3} * \frac{34}{44} * 3 = 0.7727$$

$$ Std[3P_i] = \sqrt {3Var[P_i]} = \sqrt {3*(E[P_i^2] - E[P_i]^2)} = \sqrt {3*(\frac{1}{9}*\frac{34}{44} - (\frac{1}{3} * \frac{34}{44})^2)} = 0.2420 $$


### 2(c) 

Without replacement,

The standard deviation of the run without replacement is smaller. It is because without replacement, the next run becomes more predictable. 

$$ E[P] = \sum_{i=0}^3 \frac{\binom{34}{i}\binom{10}{3-i}}{\binom{44}{3}} \frac{i}{3} = 0.7727 $$

$$ Std[P] = \sqrt{Var[P]} = \sqrt {\sum_{i=0}^3 \frac{\binom{34}{i}\binom{10}{3-i}}{\binom{44}{3}} \bigg( \frac{i}{3} - E[P] \bigg)^2}= 0.2363 $$